In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Fetal-Health-Classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    model_path: Path
    train_path: Path 

In [3]:
from FetalHealthC.constants import *
from FetalHealthC.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer 

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            train_path = config.train_path
        )
        
        return model_trainer_config

In [11]:
from sklearn.svm import SVC
import pandas as pd
from FetalHealthC.logging import logger

from FetalHealthC.utils.common import save_object

class ModelTrainer:
    def __init__(
            self,
            config: ModelTrainerConfig
            ):
        self.config = config
    
    def initiate_model_training(self):
        # Fetching the training data
        data = pd.read_csv(self.config.train_path)
        logger.info("Training Dataset has been fetched.")

        # Splitting the data 
        x_train = data.drop(columns=['fetal_health'], axis=1)
        y_train = data['fetal_health']
        logger.info("Training Dataset has been splitted.")

        # Training the model
        svc_model = SVC(C=80,degree=40)
        svc_model.fit(x_train, y_train)
        logger.info("Model has been trained.")
        
        # Saving the model
        save_object(Path(self.config.model_path), svc_model)
        logger.info("Model has been saved.")


In [13]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.initiate_model_training()
except Exception as e:
    raise e


[2024-01-13 14:48:28,759: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-13 14:48:28,759: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-13 14:48:28,769: INFO: common: created directory at: artifacts]
[2024-01-13 14:48:28,773: INFO: common: created directory at: artifacts/model_trainer]
[2024-01-13 14:48:28,805: INFO: 92992970: Training Dataset has been fetched.]
[2024-01-13 14:48:28,809: INFO: 92992970: Training Dataset has been splitted.]


[2024-01-13 14:48:28,925: INFO: 92992970: Model has been trained.]
[2024-01-13 14:48:28,925: INFO: 92992970: Model has been saved.]
